In [1]:
from yelpapi import YelpAPI
import json
import pandas as pd
import geopandas
from azure.cosmos import CosmosClient

In [2]:
cosmos_url = 'https://enca-eats-cosmos.documents.azure.com:443/'
cosmos_key = 'pAu4Vbd3QCBsOy3FdN9Cq5hTAeMMTnITZODR4QCA5j1O3bOtcRTQpV9kbLicNcZXlIBVcVhkHeSt1BqyZE4rJw=='
client = CosmosClient(cosmos_url, credential = cosmos_key)
database_name = 'enca-eats'
database = client.get_database_client(database_name)
container_name = 'restaurants'
container = database.get_container_client(container_name)

# for i in range(1, 10):
#     container.upsert_item({
#             'id': 'item{0}'.format(i),
#             'productName': 'Widget',
#             'productModel': 'Model {0}'.format(i)
#         }
#     )

In [3]:
# API Parameters
with open('yelp_key.txt', 'r') as file:
    api_key = file.read().replace('\n', '')
yelp_api = YelpAPI(api_key)

In [4]:
# Funtion to map coordinates to Chicago neighborhoods
def find_neighborhood(p):
    neighborhoods = geopandas.read_file("Boundaries - Neighborhoods.geojson")
    for index, row in neighborhoods.iterrows():
        # poly_area = row['geometry'].area
        if row['geometry'].contains(p):
            return(row['pri_neigh'])
    return('Unknown')

In [5]:
# Sample 100 restaurants Yelp pages from list of categories
category_list = ['pizza', 'sandwiches', 'mexican']
# category_list = ['restaurnts', 'food']
# category_list = ['restaurnts']

bus_list = []
# cnt = 0
for cat in category_list:
    cnt = 0
    response_check = yelp_api.search_query(categories=cat, location='Chicago', limit=1)
    tot_cnt = response_check['total']
    print(cat, tot_cnt)
    while (cnt < tot_cnt) & (cnt < 100):
        resp = yelp_api.search_query(categories=cat, location='Chicago', limit=20, offset = cnt)
        for i in resp['businesses']:
            cnt+=1
            bus_list.append(i)
            # print(cat, cnt, i['id'], i['name'])

bus_ids = []
dedupe_list = []
for i in bus_list:
    if i['id'] not in bus_ids: 
        bus_ids.append(i['id'])
        dedupe_list.append(i)
bus_list = dedupe_list

pizza 548


YelpAPIError: INTERNAL_ERROR: Something went wrong internally, please try again later.

In [10]:
from shapely.geometry import Point
bus = bus_list[0]

bus['neighborhood'] = find_neighborhood(Point(bus['coordinates']['longitude'], bus['coordinates']['latitude']))
bus

{'id': '8vFJH_paXsMocmEO_KAa3w',
 'alias': 'lou-malnatis-pizzeria-chicago',
 'name': "Lou Malnati's Pizzeria",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/rUg2qmNEK2kLmmZ5hcc6mg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/lou-malnatis-pizzeria-chicago?adjust_creative=ZzZ3ffcKnwQmN827qh1vKw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZzZ3ffcKnwQmN827qh1vKw',
 'review_count': 6198,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'},
  {'alias': 'sandwiches', 'title': 'Sandwiches'}],
 'rating': 4.0,
 'coordinates': {'latitude': 41.890357, 'longitude': -87.633704},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '439 N Wells St',
  'address2': '',
  'address3': '',
  'city': 'Chicago',
  'zip_code': '60654',
  'country': 'US',
  'state': 'IL',
  'display_address': ['439 N Wells St', 'Chicago, IL 60654']},
 'phone': '+13128289800',
 'display_phone': '(312)

In [11]:
container.upsert_item(bus)

{'id': '8vFJH_paXsMocmEO_KAa3w',
 'alias': 'lou-malnatis-pizzeria-chicago',
 'name': "Lou Malnati's Pizzeria",
 'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/rUg2qmNEK2kLmmZ5hcc6mg/o.jpg',
 'is_closed': False,
 'url': 'https://www.yelp.com/biz/lou-malnatis-pizzeria-chicago?adjust_creative=ZzZ3ffcKnwQmN827qh1vKw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=ZzZ3ffcKnwQmN827qh1vKw',
 'review_count': 6198,
 'categories': [{'alias': 'pizza', 'title': 'Pizza'},
  {'alias': 'italian', 'title': 'Italian'},
  {'alias': 'sandwiches', 'title': 'Sandwiches'}],
 'rating': 4.0,
 'coordinates': {'latitude': 41.890357, 'longitude': -87.633704},
 'transactions': ['pickup', 'delivery'],
 'price': '$$',
 'location': {'address1': '439 N Wells St',
  'address2': '',
  'address3': '',
  'city': 'Chicago',
  'zip_code': '60654',
  'country': 'US',
  'state': 'IL',
  'display_address': ['439 N Wells St', 'Chicago, IL 60654']},
 'phone': '+13128289800',
 'display_phone': '(312)